In [11]:
%bookmark HOME "/home/jupyter/meta_XLM/XLM" 
%cd -b HOME

(bookmark:HOME) -> /home/jupyter/meta_XLM/XLM
/home/jupyter/meta_XLM/XLM


In [3]:
%env csv_path=/home/jupyter
%env output_dir=/home/jupyter/data
%env data_type=para
%env languages=Limbum,Ngiemboon
! python ../bible.py --csv_path $csv_path --output_dir $output_dir --data_type $data_type --languages $languages

env: csv_path=/home/jupyter/csvs
env: output_dir=/home/jupyter/data
env: data_type=para
env: languages=Limbum,Ngiemboon
Limbum - Ngiemboon
======= Read 7950 totals samples
======= Delete 50 samples
======= Save 7900 samples


In [4]:
%env PARA=True          
%env MONO=True       
                   
%env PARA_PATH=/home/jupyter/data/para/Limbum-Ngiemboon     
%env MONO_PATH=""  
%env SAME_VOCAB=True    
%env nCodes=10000             
%env shuf_n_samples=1000000   
%env threads_for_tokenizer=16 
%env test_size=10             
%env val_size=10              

# tools paths
%env TOKENIZE=tools/tokenizer_our.sh
%env LOWER_REMOVE_ACCENT=tools/lowercase_and_remove_accent.py
%env FASTBPE=tools/fastBPE/fast


%env OUTPATH=/home/jupyter/models/africa/cluster3/data/Limbum_Ngiemboon/processed
# create output path
! mkdir -p $OUTPATH

! chmod +x $FASTBPE
! chmod +x ../build_meta_data.sh
! chmod +x tools/mosesdecoder/scripts/tokenizer/*.perl

%env n_samples=-1

%env sub_task=Limbum-Ngiemboon:-1

env: PARA=True
env: MONO=True
env: PARA_PATH=/home/jupyter/data/para/Limbum-Ngiemboon
env: MONO_PATH=""
env: SAME_VOCAB=True
env: nCodes=10000
env: shuf_n_samples=1000000
env: threads_for_tokenizer=16
env: test_size=10
env: val_size=10
env: TOKENIZE=tools/tokenizer_our.sh
env: LOWER_REMOVE_ACCENT=tools/lowercase_and_remove_accent.py
env: FASTBPE=tools/fastBPE/fast
env: OUTPATH=/home/jupyter/models/africa/cluster3/data/Limbum_Ngiemboon/processed
env: n_samples=-1
env: sub_task=Limbum-Ngiemboon:-1


In [5]:
# moins de codes que les autres, sinon "fast: fastBPE/fastBPE.hpp:458: void fastBPE::readCodes(const char* ...).Assertion `codes.find(pair) == codes.end()' failed."
%env nCodes=7000 
%env add_para_data_to_mono_data=False
! ../build_meta_data.sh $sub_task $n_samples $add_para_data_to_mono_data

env: nCodes=7000
env: add_para_data_to_mono_data=False
params ok !
*** Cleaning and tokenizing Limbum-Ngiemboon data ... ***
Tokenizer Version 1.1
Language: Limbum
Number of threads: 8
*** Tokenized (+ lowercase + accent-removal) Limbum-Ngiemboon.Limbum data to /home/jupyter/data/para/Limbum-Ngiemboon/? ***
Tokenizer Version 1.1
Language: Ngiemboon
Number of threads: 8
*** Tokenized (+ lowercase + accent-removal) Limbum-Ngiemboon.Ngiemboon data to /home/jupyter/data/para/Limbum-Ngiemboon/? ***


*** split into train / valid / test ***
shuf: write error: Broken pipe
shuf: write error
shuf: write error: Broken pipe
shuf: write error
shuf: write error: Broken pipe
shuf: write error
shuf: write error: Broken pipe
shuf: write error



***build the training set for BPE tokenization (7000 codes)***


***shuf ... Generating 1000000 random permutations of training data and store result in /home/jupyter/models/africa/cluster3/data/Limbum_Ngiemboon/processed/Limbum-Ngiemboon/bpe.train***


***Lea

In [2]:
%env OUTPATH=/home/jupyter/models/africa/cluster3/data/Limbum_Ngiemboon/processed
%env exp_id=maml
# If you don't have enough RAM or swap memory, leave these three parameters to True, otherwise you may get an error like this when evaluating 
# RuntimeError: copy_if failed to synchronize: cudaErrorAssert: device-side assert triggered
%env remove_long_sentences_train=True
%env remove_long_sentences_valid=True
%env remove_long_sentences_test=True
#--remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test

env: OUTPATH=/home/jupyter/models/africa/cluster3/data/Limbum_Ngiemboon/processed
env: exp_id=maml
env: remove_long_sentences_train=True
env: remove_long_sentences_valid=True
env: remove_long_sentences_test=True


In [3]:
# le plus grand fichier à tensor_len phrases, on cherche le multiple de "batch_size" le plus proche de ce 
# nombre par valeur supérieur : epoch_size doit etre un multiple non nul de ce nombre (pour ne pas gaspiller) 

def getEpochSize(tensor_len, batch_size):
    i = tensor_len
    while True :
        if i%batch_size == 0 :
            return i//batch_size
        i = i + 1

import io

def n_lines(file_path):
    return len(io.open(file_path, encoding='UTF-8').read().split('\n'))

In [9]:
tensor_len = n_lines(file_path = "/home/jupyter/models/africa/cluster3/data/Limbum_Ngiemboon/processed/Limbum-Ngiemboon.Limbum.train")
print("tensor_len = " + str(tensor_len))

tensor_len = 6322


In [4]:
%env epoch_size=6322

%env lgs=Limbum-Ngiemboon

# limit the number of examples (-1 by default for non limitation)
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1
#--train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

# stopping criterion (if criterion does not improve 10 times)
%env stopping_criterion=_valid_mlm_ppl,10
%env eval_bleu false

env: epoch_size=6322
env: lgs=Limbum-Ngiemboon
env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: stopping_criterion=_valid_mlm_ppl,10
env: eval_bleu=false


In [ ]:
%env mlm_steps=Limbum,Ngiemboon,Limbum-Ngiemboon
%env batch_size=8
%env max_epoch=100
%env dump_path=/home/jupyter/models/africa/cluster3
! python train.py --exp_name mlm_tlm_LimbumNgiemboon --exp_id $exp_id --dump_path $dump_path --data_path $OUTPATH --lgs $lgs --clm_steps '' --mlm_steps $mlm_steps --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --batch_size $batch_size --bptt 256 --optimizer adam,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --validation_metrics _valid_mlm_ppl --stopping_criterion $stopping_criterion --eval_bleu $eval_bleu --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

env: mlm_steps=Limbum,Ngiemboon,Limbum-Ngiemboon
env: batch_size=8
env: max_epoch=100
env: dump_path=/home/jupyter/models/africa/cluster3
FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : african-translator-vm-bis-vm
INFO - 05/21/20 08:51:40 - 0:00:00 - ============ Initialized logger ============
INFO - 05/21/20 08:51:40 - 0:00:00 - accumulate_gradients: 1
                                     ae_steps: []
                                     amp: -1
                                     asm: False
                                     attention_dropout: 0.1
                                     batch_size: 8
                                     beam_size: 1
                 

INFO - 05/21/20 08:51:40 - 0:00:00 - 259414 words (6667 unique) in 6321 sentences. 0 unknown words (0 unique) covering 0.00% of the data.

INFO - 05/21/20 08:51:40 - 0:00:00 - Loading data from /home/jupyter/models/africa/cluster3/data/Limbum_Ngiemboon/processed/valid.Limbum.pth ...
INFO - 05/21/20 08:51:40 - 0:00:00 - 33215 words (6667 unique) in 790 sentences. 13 unknown words (10 unique) covering 0.04% of the data.

INFO - 05/21/20 08:51:40 - 0:00:00 - Loading data from /home/jupyter/models/africa/cluster3/data/Limbum_Ngiemboon/processed/test.Limbum.pth ...
INFO - 05/21/20 08:51:40 - 0:00:00 - 32382 words (6667 unique) in 790 sentences. 5 unknown words (4 unique) covering 0.02% of the data.

INFO - 05/21/20 08:51:40 - 0:00:00 - ============ Monolingual data (Ngiemboon)
INFO - 05/21/20 08:51:40 - 0:00:00 - Loading data from /home/jupyter/models/africa/cluster3/data/Limbum_Ngiemboon/processed/train.Ngiemboon.pth ...
INFO - 05/21/20 08:51:40 - 0:00:00 - 220240 words (6667 unique) in 63

INFO - 05/21/20 08:51:45 - 0:00:05 - Found 0 memories.
INFO - 05/21/20 08:51:45 - 0:00:05 - Found 6 FFN.
INFO - 05/21/20 08:51:45 - 0:00:05 - Found 102 parameters in model.
INFO - 05/21/20 08:51:45 - 0:00:05 - Optimizers: model
WARNING - 05/21/20 08:51:45 - 0:00:05 - Reloading checkpoint from /home/jupyter/models/africa/cluster3/mlm_tlm_LimbumNgiemboon/maml/checkpoint.pth ...
WARNING - 05/21/20 08:51:53 - 0:00:13 - Not reloading checkpoint optimizer model.
WARNING - 05/21/20 08:51:53 - 0:00:13 - No 'num_updates' for optimizer model.
WARNING - 05/21/20 08:51:53 - 0:00:13 - Checkpoint reloaded. Resuming at epoch 50 / iteration 13200 ...
INFO - 05/21/20 08:51:54 - 0:00:14 - ============ Starting epoch 50 ... ============
INFO - 05/21/20 08:51:54 - 0:00:14 - Creating new training data iterator (pred,Ngiemboon) ...
INFO - 05/21/20 08:51:55 - 0:00:15 - Creating new training data iterator (pred,Limbum) ...
INFO - 05/21/20 08:51:56 - 0:00:16 - Creating new training data iterator (pred,Limbum,N

INFO - 05/21/20 08:56:49 - 0:05:09 -   13395 -   13.81 sent/s - 15913.27 words/s - MLM-Limbum:  0.8597 || MLM-Ngiemboon:  0.6815 || MLM-Limbum-Ngiemboon:  1.1836 -  - model LR: 1.0000e-04
INFO - 05/21/20 08:56:56 - 0:05:16 -   13400 -   17.75 sent/s - 20958.31 words/s - MLM-Limbum:  0.8296 || MLM-Ngiemboon:  0.6267 || MLM-Limbum-Ngiemboon:  1.2779 -  - model LR: 1.0000e-04
INFO - 05/21/20 08:57:04 - 0:05:24 -   13405 -   13.70 sent/s - 16585.57 words/s - MLM-Limbum:  0.8133 || MLM-Ngiemboon:  0.6954 || MLM-Limbum-Ngiemboon:  0.9922 -  - model LR: 1.0000e-04
INFO - 05/21/20 08:57:12 - 0:05:32 -   13410 -   14.96 sent/s - 18555.99 words/s - MLM-Limbum:  0.7716 || MLM-Ngiemboon:  0.6955 || MLM-Limbum-Ngiemboon:  1.1704 -  - model LR: 1.0000e-04
INFO - 05/21/20 08:57:20 - 0:05:40 - Creating new training data iterator (pred,Ngiemboon) ...
INFO - 05/21/20 08:57:20 - 0:05:40 -   13415 -   15.42 sent/s - 19562.18 words/s - MLM-Limbum:  0.8130 || MLM-Ngiemboon:  0.7309 || MLM-Limbum-Ngiemboon: 

INFO - 05/21/20 09:00:44 - 0:09:04 -   13540 -   19.24 sent/s - 38479.57 words/s - MLM-Limbum:  0.8684 || MLM-Ngiemboon:  0.6658 || MLM-Limbum-Ngiemboon:  1.3950 -  - model LR: 1.0000e-04
INFO - 05/21/20 09:00:51 - 0:09:11 -   13545 -   19.17 sent/s - 38898.88 words/s - MLM-Limbum:  0.7917 || MLM-Ngiemboon:  0.7184 || MLM-Limbum-Ngiemboon:  1.1036 -  - model LR: 1.0000e-04
INFO - 05/21/20 09:00:57 - 0:09:17 -   13550 -   19.83 sent/s - 40784.88 words/s - MLM-Limbum:  0.8515 || MLM-Ngiemboon:  0.7306 || MLM-Limbum-Ngiemboon:  1.0197 -  - model LR: 1.0000e-04
INFO - 05/21/20 09:01:03 - 0:09:23 -   13555 -   18.94 sent/s - 39531.70 words/s - MLM-Limbum:  0.7230 || MLM-Ngiemboon:  0.6158 || MLM-Limbum-Ngiemboon:  1.1483 -  - model LR: 1.0000e-04
INFO - 05/21/20 09:01:09 - 0:09:29 -   13560 -   18.45 sent/s - 39030.48 words/s - MLM-Limbum:  0.7820 || MLM-Ngiemboon:  0.6694 || MLM-Limbum-Ngiemboon:  1.1459 -  - model LR: 1.0000e-04
INFO - 05/21/20 09:01:15 - 0:09:35 -   13565 -   19.84 sent/

...

INFO - 05/21/20 09:26:48 - 0:35:08 - Creating new training data iterator (pred,Ngiemboon) ...
INFO - 05/21/20 09:26:48 - 0:35:08 -   14610 -   18.77 sent/s - 155555.33 words/s - MLM-Limbum:  0.7060 || MLM-Ngiemboon:  0.5898 || MLM-Limbum-Ngiemboon:  1.3469 -  - model LR: 1.0000e-04
INFO - 05/21/20 09:26:55 - 0:35:15 -   14615 -   18.25 sent/s - 151792.66 words/s - MLM-Limbum:  0.7856 || MLM-Ngiemboon:  0.5838 || MLM-Limbum-Ngiemboon:  1.1701 -  - model LR: 1.0000e-04
INFO - 05/21/20 09:27:01 - 0:35:21 -   14620 -   19.90 sent/s - 166069.05 words/s - MLM-Limbum:  0.6222 || MLM-Ngiemboon:  0.5911 || MLM-Limbum-Ngiemboon:  1.0745 -  - model LR: 1.0000e-04
INFO - 05/21/20 09:27:07 - 0:35:27 -   14625 -   20.03 sent/s - 167718.27 words/s - MLM-Limbum:  0.7845 || MLM-Ngiemboon:  0.6250 || MLM-Limbum-Ngiemboon:  1.2330 -  - model LR: 1.0000e-04
INFO - 05/21/20 09:27:13 - 0:35:33 -   14630 -   19.04 sent/s - 160055.64 words/s - MLM-Limbum:  0.7754 || MLM-Ngiemboon:  0.6311 || MLM-Limbum-Ngiemb

INFO - 05/21/20 09:30:45 - 0:39:05 - ============ garbage collector collecting 0 ...
INFO - 05/21/20 09:30:45 - 0:39:05 - ============ Starting epoch 56 ... ============
INFO - 05/21/20 09:30:47 - 0:39:07 -   14785 -    6.61 sent/s - 61610.90 words/s - MLM-Limbum:  0.6704 || MLM-Ngiemboon:  0.6338 || MLM-Limbum-Ngiemboon:  1.0548 -  - model LR: 1.0000e-04
INFO - 05/21/20 09:30:54 - 0:39:14 -   14790 -   16.18 sent/s - 151177.14 words/s - MLM-Limbum:  0.7542 || MLM-Ngiemboon:  0.5891 || MLM-Limbum-Ngiemboon:  1.2576 -  - model LR: 1.0000e-04
INFO - 05/21/20 09:31:02 - 0:39:22 -   14795 -   15.41 sent/s - 144460.65 words/s - MLM-Limbum:  0.7038 || MLM-Ngiemboon:  0.5363 || MLM-Limbum-Ngiemboon:  1.1906 -  - model LR: 1.0000e-04
INFO - 05/21/20 09:31:09 - 0:39:29 -   14800 -   16.36 sent/s - 153808.89 words/s - MLM-Limbum:  0.7146 || MLM-Ngiemboon:  0.5493 || MLM-Limbum-Ngiemboon:  1.0216 -  - model LR: 1.0000e-04
INFO - 05/21/20 09:31:10 - 0:39:30 - Creating new training data iterator (p

INFO - 05/21/20 09:49:41 - 0:58:01 -   15530 -   19.74 sent/s - 270345.84 words/s - MLM-Limbum:  0.7102 || MLM-Ngiemboon:  0.5910 || MLM-Limbum-Ngiemboon:  1.0543 -  - model LR: 1.0000e-04
INFO - 05/21/20 09:49:48 - 0:58:08 -   15535 -   18.65 sent/s - 256001.24 words/s - MLM-Limbum:  0.6996 || MLM-Ngiemboon:  0.5454 || MLM-Limbum-Ngiemboon:  1.2467 -  - model LR: 1.0000e-04
INFO - 05/21/20 09:49:54 - 0:58:14 -   15540 -   18.40 sent/s - 253119.34 words/s - MLM-Limbum:  0.7669 || MLM-Ngiemboon:  0.6409 || MLM-Limbum-Ngiemboon:  1.1431 -  - model LR: 1.0000e-04
INFO - 05/21/20 09:50:00 - 0:58:21 -   15545 -   18.60 sent/s - 256381.62 words/s - MLM-Limbum:  0.7348 || MLM-Ngiemboon:  0.5144 || MLM-Limbum-Ngiemboon:  1.0425 -  - model LR: 1.0000e-04
INFO - 05/21/20 09:50:07 - 0:58:27 -   15550 -   18.72 sent/s - 258614.69 words/s - MLM-Limbum:  0.6965 || MLM-Ngiemboon:  0.5955 || MLM-Limbum-Ngiemboon:  1.0384 -  - model LR: 1.0000e-04
INFO - 05/21/20 09:50:14 - 0:58:34 -   15555 -   18.00 

INFO - 05/21/20 09:52:52 - 1:01:12 -   15675 -   18.39 sent/s - 267477.14 words/s - MLM-Limbum:  0.6636 || MLM-Ngiemboon:  0.6067 || MLM-Limbum-Ngiemboon:  1.1053 -  - model LR: 1.0000e-04
INFO - 05/21/20 09:52:59 - 1:01:19 -   15680 -   18.84 sent/s - 274573.28 words/s - MLM-Limbum:  0.6948 || MLM-Ngiemboon:  0.5853 || MLM-Limbum-Ngiemboon:  1.0470 -  - model LR: 1.0000e-04
INFO - 05/21/20 09:53:05 - 1:01:25 -   15685 -   18.05 sent/s - 263606.77 words/s - MLM-Limbum:  0.6734 || MLM-Ngiemboon:  0.6150 || MLM-Limbum-Ngiemboon:  1.2864 -  - model LR: 1.0000e-04
INFO - 05/21/20 09:53:11 - 1:01:31 -   15690 -   19.06 sent/s - 278906.81 words/s - MLM-Limbum:  0.5860 || MLM-Ngiemboon:  0.6159 || MLM-Limbum-Ngiemboon:  1.0456 -  - model LR: 1.0000e-04
INFO - 05/21/20 09:53:16 - 1:01:36 -   15695 -   28.53 sent/s - 418257.99 words/s - MLM-Limbum:  0.7332 || MLM-Ngiemboon:  0.5174 || MLM-Limbum-Ngiemboon:  1.1348 -  - model LR: 1.0000e-04
INFO - 05/21/20 09:53:20 - 1:01:40 -   15700 -   28.75 

INFO - 05/21/20 09:57:03 - 1:05:23 - ============ garbage collector collecting 0 ...
INFO - 05/21/20 09:57:03 - 1:05:23 - ============ Starting epoch 60 ... ============
INFO - 05/21/20 09:57:12 - 1:05:32 -   15845 -    6.67 sent/s - 103716.31 words/s - MLM-Limbum:  0.6885 || MLM-Ngiemboon:  0.6005 || MLM-Limbum-Ngiemboon:  1.1026 -  - model LR: 1.0000e-04
INFO - 05/21/20 09:57:21 - 1:05:41 -   15850 -   14.17 sent/s - 220726.43 words/s - MLM-Limbum:  0.7791 || MLM-Ngiemboon:  0.5574 || MLM-Limbum-Ngiemboon:  1.0275 -  - model LR: 1.0000e-04
INFO - 05/21/20 09:57:29 - 1:05:49 -   15855 -   15.09 sent/s - 235545.38 words/s - MLM-Limbum:  0.6179 || MLM-Ngiemboon:  0.6028 || MLM-Limbum-Ngiemboon:  1.2603 -  - model LR: 1.0000e-04
INFO - 05/21/20 09:57:29 - 1:05:49 - Creating new training data iterator (pred,Limbum) ...
INFO - 05/21/20 09:57:37 - 1:05:57 -   15860 -   13.90 sent/s - 217363.41 words/s - MLM-Limbum:  0.6582 || MLM-Ngiemboon:  0.5642 || MLM-Limbum-Ngiemboon:  1.0629 -  - mode

**Train a (unsupervised/supervised) MT from a pretrained meta-model**

In [12]:
%env epoch_size=6360
%env lgs=Limbum-Ngiemboon

env: epoch_size=6360
env: lgs=Limbum-Ngiemboon


In [13]:
# limit the number of examples (-1 by default for non limitation)
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1
#--train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

%env eval_bleu=true
! chmod +x src/evaluation/multi-bleu.perl

env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: eval_bleu=true


In [14]:
%env stopping_criterion=valid_Limbum-Ngiemboon_mt_bleu,10
%env validation_metrics=valid_Limbum-Ngiemboon_mt_bleu

%env reload_model=/home/jupyter/models/africa/cluster3/mlm_tlm_LimbumNgiemboon/maml/best-valid_mlm_ppl.pth,/home/jupyter/models/africa/cluster3/mlm_tlm_LimbumNgiemboon/maml/best-valid_mlm_ppl.pth

%env ae_steps=Limbum,Ngiemboon
%env bt_steps=Limbum-Ngiemboon-Limbum,Ngiemboon-Limbum-Ngiemboon

env: stopping_criterion=valid_Limbum-Ngiemboon_mt_bleu,10
env: validation_metrics=valid_Limbum-Ngiemboon_mt_bleu
env: reload_model=/home/jupyter/models/africa/cluster3/mlm_tlm_LimbumNgiemboon/maml/best-valid_mlm_ppl.pth,/home/jupyter/models/africa/cluster3/mlm_tlm_LimbumNgiemboon/maml/best-valid_mlm_ppl.pth
env: ae_steps=Limbum,Ngiemboon
env: bt_steps=Limbum-Ngiemboon-Limbum,Ngiemboon-Limbum-Ngiemboon


In [16]:
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1

%env max_epoch=100

# unsupervised MT
#! python train.py --exp_name UnSupMT_LimbumNgiemboon --exp_id $exp_id --dump_path ./dumped/ --reload_model $reload_model --data_path $OUTPATH --lgs $lgs --ae_steps $ae_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test    

env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: max_epoch=100


In [ ]:
#! rsync -av --progress /content/meta_XLM/XLM/dumped/mlm_tlm_BafiaBulu  /content/drive/"My Drive"/African_Translator/models/africa/cluster1

In [18]:
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1

%env max_epoch=100
%env batch_size=32
# supervised MT
%env mt_steps=Limbum-Ngiemboon,Ngiemboon-Limbum          
! python train.py --exp_name SupMT_LimbumNgiemboon --exp_id $exp_id  --dump_path ./dumped/ --reload_model $reload_model --data_path $OUTPATH --lgs $lgs --ae_steps $ae_steps --mt_steps $mt_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test    

env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: max_epoch=100
env: batch_size=32
env: mt_steps=Limbum-Ngiemboon,Ngiemboon-Limbum
FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : african-translator-vm-bis-vm
INFO - 05/22/20 04:38:39 - 0:00:00 - ============ Initialized logger ============
INFO - 05/22/20 04:38:39 - 0:00:00 - accumulate_gradients: 1
                                     ae_steps: ['Limbum', 'Ngiemboon']
                                     amp: -1
                                     asm: False
                                     attention_dropout: 0.1
                                     batch_size: 32
                          

INFO - 05/22/20 04:38:39 - 0:00:00 - 220240 words (6667 unique) in 6321 sentences. 0 unknown words (0 unique) covering 0.00% of the data.
INFO - 05/22/20 04:38:39 - 0:00:00 - Removed 0 empty sentences.
INFO - 05/22/20 04:38:39 - 0:00:00 - Removed 6 too long sentences.

INFO - 05/22/20 04:38:39 - 0:00:00 - Loading data from /home/jupyter/models/africa/cluster3/data/Limbum_Ngiemboon/processed/valid.Ngiemboon.pth ...
INFO - 05/22/20 04:38:39 - 0:00:00 - 28350 words (6667 unique) in 790 sentences. 21 unknown words (16 unique) covering 0.07% of the data.
INFO - 05/22/20 04:38:39 - 0:00:00 - Removed 0 too long sentences.

INFO - 05/22/20 04:38:39 - 0:00:00 - Loading data from /home/jupyter/models/africa/cluster3/data/Limbum_Ngiemboon/processed/test.Ngiemboon.pth ...
INFO - 05/22/20 04:38:39 - 0:00:00 - 27928 words (6667 unique) in 790 sentences. 19 unknown words (14 unique) covering 0.07% of the data.
INFO - 05/22/20 04:38:39 - 0:00:00 - Removed 1 too long sentences.


INFO - 05/22/20 04:38:

INFO - 05/22/20 04:38:44 - 0:00:05 - Number of parameters (encoder): 82939403
INFO - 05/22/20 04:38:44 - 0:00:05 - Number of parameters (decoder): 108142091
INFO - 05/22/20 04:38:45 - 0:00:06 - Found 0 memories.
INFO - 05/22/20 04:38:45 - 0:00:06 - Found 12 FFN.
INFO - 05/22/20 04:38:45 - 0:00:06 - Found 264 parameters in model.
INFO - 05/22/20 04:38:45 - 0:00:06 - Optimizers: model
INFO - 05/22/20 04:38:45 - 0:00:06 - ============ Starting epoch 0 ... ============
INFO - 05/22/20 04:38:45 - 0:00:06 - Creating new training data iterator (ae,Limbum) ...
INFO - 05/22/20 04:38:46 - 0:00:07 - Creating new training data iterator (ae,Ngiemboon) ...
INFO - 05/22/20 04:38:47 - 0:00:08 - Creating new training data iterator (mt,Ngiemboon,Limbum) ...
INFO - 05/22/20 04:38:48 - 0:00:08 - Creating new training data iterator (mt,Limbum,Ngiemboon) ...
INFO - 05/22/20 04:38:48 - 0:00:09 - Creating new training data iterator (bt,Limbum) ...
INFO - 05/22/20 04:38:49 - 0:00:10 - Creating new training dat

INFO - 05/22/20 04:54:03 - 0:15:24 -      60 -   17.93 sent/s -  8269.52 words/s - AE-Limbum:  3.5242 || AE-Ngiemboon:  4.4757 || MT-Limbum-Ngiemboon:  4.3364 || MT-Ngiemboon-Limbum:  3.5756 || BT-Limbum-Ngiemboon-Limbum:  3.6682 || BT-Ngiemboon-Limbum-Ngiemboon:  4.4454 -  - model LR: 9.0910e-06
INFO - 05/22/20 04:55:11 - 0:16:32 -      65 -   18.16 sent/s -  9322.59 words/s - AE-Limbum:  3.4915 || AE-Ngiemboon:  4.2906 || MT-Limbum-Ngiemboon:  4.3012 || MT-Ngiemboon-Limbum:  3.5450 || BT-Limbum-Ngiemboon-Limbum:  3.5129 || BT-Ngiemboon-Limbum-Ngiemboon:  4.3929 -  - model LR: 9.8403e-06
INFO - 05/22/20 04:55:51 - 0:17:12 - ============ End of epoch 1 ============
INFO - 05/22/20 04:57:13 - 0:18:34 - BLEU ./dumped/SupMT_LimbumNgiemboon/maml/hypotheses/hyp1.Limbum-Ngiemboon.valid.txt ./dumped/SupMT_LimbumNgiemboon/maml/hypotheses/ref.Limbum-Ngiemboon.valid.txt : 1.530000
INFO - 05/22/20 04:58:35 - 0:19:55 - BLEU ./dumped/SupMT_LimbumNgiemboon/maml/hypotheses/hyp1.Ngiemboon-Limbum.valid

INFO - 05/22/20 05:14:15 - 0:35:36 - New best validation score: 2.080000
INFO - 05/22/20 05:14:15 - 0:35:36 - Saving checkpoint to ./dumped/SupMT_LimbumNgiemboon/maml/checkpoint.pth ...
WARNING - 05/22/20 05:14:15 - 0:35:36 - Saving encoder parameters ...
WARNING - 05/22/20 05:14:15 - 0:35:36 - Saving decoder parameters ...
WARNING - 05/22/20 05:14:15 - 0:35:36 - Saving model optimizer ...
INFO - 05/22/20 05:14:34 - 0:35:55 - ============ garbage collector collecting 0 ...
INFO - 05/22/20 05:14:34 - 0:35:55 - ============ Starting epoch 3 ... ============
INFO - 05/22/20 05:15:17 - 0:36:38 -     105 -    3.23 sent/s -  2498.45 words/s - AE-Limbum:  2.9079 || AE-Ngiemboon:  3.6810 || MT-Limbum-Ngiemboon:  3.6284 || MT-Ngiemboon-Limbum:  2.9755 || BT-Limbum-Ngiemboon-Limbum:  3.0488 || BT-Ngiemboon-Limbum-Ngiemboon:  3.9096 -  - model LR: 1.5834e-05
INFO - 05/22/20 05:16:14 - 0:37:35 -     110 -   20.63 sent/s - 18826.34 words/s - AE-Limbum:  2.8897 || AE-Ngiemboon:  3.6190 || MT-Limbum-

...

INFO - 05/22/20 09:53:07 - 5:14:28 -     900 -   45.65 sent/s - 297326.29 words/s - AE-Limbum:  0.4578 || AE-Ngiemboon:  0.3413 || MT-Limbum-Ngiemboon:  0.9398 || MT-Ngiemboon-Limbum:  1.0135 || BT-Limbum-Ngiemboon-Limbum:  1.2077 || BT-Ngiemboon-Limbum-Ngiemboon:  1.0046 -  - model LR: 8.6066e-05
INFO - 05/22/20 09:53:36 - 5:14:57 -     905 -   46.79 sent/s - 297067.68 words/s - AE-Limbum:  0.4235 || AE-Ngiemboon:  0.3893 || MT-Limbum-Ngiemboon:  1.0011 || MT-Ngiemboon-Limbum:  1.0073 || BT-Limbum-Ngiemboon-Limbum:  1.1658 || BT-Ngiemboon-Limbum-Ngiemboon:  0.8235 -  - model LR: 8.5828e-05
INFO - 05/22/20 09:54:07 - 5:15:27 -     910 -   40.65 sent/s - 290870.29 words/s - AE-Limbum:  0.4177 || AE-Ngiemboon:  0.3456 || MT-Limbum-Ngiemboon:  0.9310 || MT-Ngiemboon-Limbum:  1.0282 || BT-Limbum-Ngiemboon-Limbum:  1.1112 || BT-Ngiemboon-Limbum-Ngiemboon:  0.9223 -  - model LR: 8.5592e-05
INFO - 05/22/20 09:54:36 - 5:15:56 -     915 -   41.44 sent/s - 305273.52 words/s - AE-Limbum:  0.4227 

INFO - 05/22/20 10:01:57 - 5:23:18 - ============ garbage collector collecting 0 ...
INFO - 05/22/20 10:01:57 - 5:23:18 - ============ Starting epoch 28 ... ============
INFO - 05/22/20 10:01:59 - 5:23:20 - Creating new training data iterator (mt,Limbum,Ngiemboon) ...
INFO - 05/22/20 10:02:18 - 5:23:39 -     955 -    8.69 sent/s - 65337.55 words/s - AE-Limbum:  0.3569 || AE-Ngiemboon:  0.2798 || MT-Limbum-Ngiemboon:  0.6613 || MT-Ngiemboon-Limbum:  0.9847 || BT-Limbum-Ngiemboon-Limbum:  0.9845 || BT-Ngiemboon-Limbum-Ngiemboon:  0.7560 -  - model LR: 8.3551e-05
INFO - 05/22/20 10:02:49 - 5:24:10 -     960 -   39.01 sent/s - 301570.05 words/s - AE-Limbum:  0.3654 || AE-Ngiemboon:  0.2642 || MT-Limbum-Ngiemboon:  0.5877 || MT-Ngiemboon-Limbum:  0.9806 || BT-Limbum-Ngiemboon-Limbum:  0.9541 || BT-Ngiemboon-Limbum-Ngiemboon:  0.7538 -  - model LR: 8.3333e-05
INFO - 05/22/20 10:03:18 - 5:24:39 -     965 -   45.15 sent/s - 322201.31 words/s - AE-Limbum:  0.3153 || AE-Ngiemboon:  0.2664 || MT-

INFO - 05/22/20 10:20:26 - 5:41:47 - BLEU ./dumped/SupMT_LimbumNgiemboon/maml/hypotheses/hyp29.Ngiemboon-Limbum.test.txt ./dumped/SupMT_LimbumNgiemboon/maml/hypotheses/ref.Ngiemboon-Limbum.test.txt : 10.440000
INFO - 05/22/20 10:20:26 - 5:41:47 - epoch -> 29.000000
INFO - 05/22/20 10:20:26 - 5:41:47 - valid_Limbum-Ngiemboon_mt_ppl -> 77.411538
INFO - 05/22/20 10:20:26 - 5:41:47 - valid_Limbum-Ngiemboon_mt_acc -> 35.645178
INFO - 05/22/20 10:20:26 - 5:41:47 - valid_Limbum-Ngiemboon_mt_bleu -> 6.210000
INFO - 05/22/20 10:20:26 - 5:41:47 - valid_Ngiemboon-Limbum_mt_ppl -> 18.533022
INFO - 05/22/20 10:20:26 - 5:41:47 - valid_Ngiemboon-Limbum_mt_acc -> 45.353171
INFO - 05/22/20 10:20:26 - 5:41:47 - valid_Ngiemboon-Limbum_mt_bleu -> 10.750000
INFO - 05/22/20 10:20:26 - 5:41:47 - test_Limbum-Ngiemboon_mt_ppl -> 72.896025
INFO - 05/22/20 10:20:26 - 5:41:47 - test_Limbum-Ngiemboon_mt_acc -> 36.628559
INFO - 05/22/20 10:20:26 - 5:41:47 - test_Limbum-Ngiemboon_mt_bleu -> 5.940000
INFO - 05/22/20 

INFO - 05/22/20 10:27:33 - 5:48:54 -    1065 -   38.76 sent/s - 339411.96 words/s - AE-Limbum:  0.3060 || AE-Ngiemboon:  0.2099 || MT-Limbum-Ngiemboon:  0.5689 || MT-Ngiemboon-Limbum:  0.5603 || BT-Limbum-Ngiemboon-Limbum:  0.8817 || BT-Ngiemboon-Limbum-Ngiemboon:  0.5159 -  - model LR: 7.9119e-05
INFO - 05/22/20 10:28:03 - 5:49:24 -    1070 -   38.96 sent/s - 344852.66 words/s - AE-Limbum:  0.2864 || AE-Ngiemboon:  0.1966 || MT-Limbum-Ngiemboon:  0.5030 || MT-Ngiemboon-Limbum:  0.5774 || BT-Limbum-Ngiemboon-Limbum:  0.8421 || BT-Ngiemboon-Limbum-Ngiemboon:  0.5131 -  - model LR: 7.8934e-05
INFO - 05/22/20 10:28:15 - 5:49:36 - Creating new training data iterator (ae,Limbum) ...
INFO - 05/22/20 10:28:18 - 5:49:39 - Creating new training data iterator (bt,Limbum) ...
INFO - 05/22/20 10:28:33 - 5:49:54 -    1075 -   45.50 sent/s - 352582.22 words/s - AE-Limbum:  0.2729 || AE-Ngiemboon:  0.1886 || MT-Limbum-Ngiemboon:  0.5554 || MT-Ngiemboon-Limbum:  0.5612 || BT-Limbum-Ngiemboon-Limbum:  

INFO - 05/22/20 10:37:17 - 5:58:37 - ============ garbage collector collecting 0 ...
INFO - 05/22/20 10:37:17 - 5:58:37 - ============ Starting epoch 33 ... ============
INFO - 05/22/20 10:37:35 - 5:58:55 -    1125 -    9.17 sent/s - 73893.90 words/s - AE-Limbum:  0.2494 || AE-Ngiemboon:  0.2104 || MT-Limbum-Ngiemboon:  0.4553 || MT-Ngiemboon-Limbum:  0.5344 || BT-Limbum-Ngiemboon-Limbum:  0.6909 || BT-Ngiemboon-Limbum-Ngiemboon:  0.6253 -  - model LR: 7.6980e-05
INFO - 05/22/20 10:38:06 - 5:59:26 -    1130 -   45.79 sent/s - 353974.35 words/s - AE-Limbum:  0.2557 || AE-Ngiemboon:  0.1948 || MT-Limbum-Ngiemboon:  0.5085 || MT-Ngiemboon-Limbum:  0.5076 || BT-Limbum-Ngiemboon-Limbum:  0.7270 || BT-Ngiemboon-Limbum-Ngiemboon:  0.5327 -  - model LR: 7.6810e-05
INFO - 05/22/20 10:38:35 - 5:59:56 -    1135 -   39.78 sent/s - 374019.52 words/s - AE-Limbum:  0.2361 || AE-Ngiemboon:  0.1640 || MT-Limbum-Ngiemboon:  0.5116 || MT-Ngiemboon-Limbum:  0.5301 || BT-Limbum-Ngiemboon-Limbum:  0.7329 ||

In [ ]:
#! rsync -av --progress /content/meta_XLM/XLM/dumped/mlm_tlm_BafiaBulu  /content/drive/"My Drive"/African_Translator/models/africa/cluster1